In [17]:
import pandas as pd;
import plotly.graph_objects as go

# https://epistat.wiv-isp.be/covid/
# Load "Hospitalisations by date and provinces" (c) Sciensano
hospitalisations = pd.read_csv('https://epistat.sciensano.be/Data/COVID19BE_HOSP.csv')  

hospfla = hospitalisations.query('REGION == "Flanders"').groupby(by="DATE")
totalwithouticu = (hospfla['TOTAL_IN'].sum()-hospfla['TOTAL_IN_ICU'].sum())

totalincu = hospfla['TOTAL_IN_ICU'].sum()

fig = go.Figure( data=[
    go.Bar(name='Totaal niet-intensieve zorg', x=totalwithouticu.index, y=totalwithouticu.values),
    go.Bar(name='Totaal intensieve zorg', x=totalincu.index, y=totalincu.values)
])
# Change the bar mode
fig.update_layout(barmode='stack',title='Covid-19 hospitalisaties in Vlaanderen')
fig.show(renderer="iframe")
